avoid any additional training and directly print results 
- determine whether the models (CPC or TS) are already capable of generating meaningful representations after the initial unsupervised/self-supervised training.

In [1]:
import wandb
from linearclassifier_all import train_extract, test_extract, linearclassifier, Wearable
import itertools
import torch
from torch.utils.data import DataLoader


from CPC import CPC
from TS import TS

In [2]:
def no_train(model, type, train_loader, test_loader, device):
# def train(num_epochs, model, type, optimizer, train_loader, test_loader, device, test_every=1):

    # Directly extract representations without training
    train_c_t, train_label, train_loss = train_extract(model, train_loader, device, optimizer=None)  # Pass None for optimizer
    test_c_t, test_label, test_loss = test_extract(model, test_loader, device)

    # Evaluate linear classifier without updates
    accuracy = linearclassifier(type, train_c_t, train_label, test_c_t, test_label, log_graph=False)    # AUC-ROC, precision, F1 score, etc are saved in wandb already by the linearclassifier function
    print(f"Initial Accuracy without Training: {accuracy * 100:.2f}%")

    # Log metrics directly
    wandb.log({"initial_accuracy": accuracy})


In [3]:
modelname=['CPC', 'TS']
type=['ClassifierNN', 'LogisticRegression']
#

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = [256,512,1024,2048]

path = '/home/ria/MMBSintern/bachelor/exercise/data' 

train_data = Wearable(path, True)
test_data = Wearable(path, False)


train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=0)
test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=False, num_workers=0) 



# Generate all combinations of modelname and batchsize
combinations = list(itertools.product(modelname, type, batch_size))

# Run each combination with each value of num_epochs

for modelname, type, bs in combinations:
        
    print(f"Running with model: {modelname}, classification:{type}, batch size: {bs}")
        
    if modelname == 'CPC':
        model = CPC(timestep=3, batch_size=bs, seq_len=200).to(device)

    elif modelname == 'TS':
        model = TS(timestep=3, batch_size=bs, seq_len=200).to(device)

    else:
        raise ValueError(f"Unknown model name: {modelname}")

        
    config = dict( model = modelname,
                    classification = type,
                    batch_size = bs
                    )
                
            
    wandb.init(project="LinearClassification no model-train, 2 types, w dropout", 
                name=f"{modelname,type,bs}",
                config=config,
                reinit=True)


    stats = no_train(model, config['classification'], train_loader, test_loader, device)
            
    wandb.finish() 
            

Running with model: CPC, classification:ClassifierNN, batch size: 256


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: riaxchoi (riaxchoi-heinrich-heine-university-d-sseldorf). Use `wandb login --relogin` to force relogin


/home/ria/miniconda3/envs/AILS/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Attempt to open cnn_infer failed: handle=0 error: libcudnn_cnn_infer.so.8: cannot open shared object file: No such file or directory (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:81.)
  return F.conv1d(input, weight, bias, self.stride,


Initial Accuracy without Training: 58.45%


ROC-AUC Score,▁
accuracy,▁
f1_AFib,▁
f1_non-AFib,▁
initial_accuracy,▁
precision_AFib,▁
precision_non-AFib,▁
recall_AFib,▁
recall_non-AFib,▁
ROC-AUC Score,0.69079
accuracy,0.58449


...

Initial Accuracy without Training: 67.81%


ROC-AUC Score,▁
accuracy,▁
f1_AFib,▁
f1_non-AFib,▁
initial_accuracy,▁
precision_AFib,▁
precision_non-AFib,▁
recall_AFib,▁
recall_non-AFib,▁
ROC-AUC Score,0.75771
accuracy,0.67812


Running with model: TS, classification:LogisticRegression, batch size: 2048


Initial Accuracy without Training: 70.75%


ROC-AUC Score,▁
accuracy,▁
f1_AFib,▁
f1_non-AFib,▁
initial_accuracy,▁
precision_AFib,▁
precision_non-AFib,▁
recall_AFib,▁
recall_non-AFib,▁
ROC-AUC Score,0.80201
accuracy,0.70748
